# DAY-AHEAD CONDITIONAL LINEAR MODEL

## 2025 ENTSO-E data BZN|NL

Residual load x price correlation depending on hour, season and market regime

In [4]:
#Load data from baseline models

%store -r price_load
%store -r morning_price_load
%store -r afternoon_price_load
%store -r night_price_load
%store -r spring_price_load
%store -r summer_price_load
%store -r autumn_price_load
%store -r winter_price_load

price_load

,Residual load actual (MW),Residual load day-ahead (MW),Price,Month,Hour,season
0,6217.86,6199.38,13.62,1,0,winter
1,6143.04,6151.90,13.62,1,0,winter
2,6408.02,6004.90,13.62,1,0,winter
3,6269.17,5863.26,13.62,1,0,winter
4,6676.54,5754.04,6.24,1,1,winter
...,...,...,...,...,...,...
35035,5766.6,8434.78,73.76,12,22,winter
35036,5731.13,8154.23,87.16,12,23,winter
35037,5754.41,8023.97,74.27,12,23,winter
35038,5731.14,7700.52,67.60,12,23,winter


In [5]:
import pandas as pd
import statsmodels.formula.api as smf
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

def month_to_season(m):
    if m in [12, 1, 2]:
        return "winter"
    elif m in [3, 4, 5]:
        return "spring"
    elif m in [6, 7, 8]:
        return "summer"
    else:
        return "autumn"

price_load["season"] = price_load["Month"].apply(month_to_season)

price_load = price_load.rename(columns={
    "Residual load day-ahead (MW)": "residual_load",
    "Price": "price"
})

price_load["hour"] = price_load["Hour"].astype("category")
price_load["season"] = price_load["season"].astype("category")

model = smf.ols(
    formula="""
    price ~ residual_load
          + C(hour)
          + C(season)
          + residual_load:C(season)
    """,
    data=price_load
).fit()

In [ ]:
#Testing block: compare my prediction model outcome for January 2025 against observed data in January 2026

#NEXT: GET DAY-AHEAD RESIDUAL LOAD FOR JAN 2026 ENTSO-E, FEED THE MODEL, COMPARE WITH OSBERVED PRICES

repo_root = Path("..").resolve()
sys.path.append(str(repo_root))
from Source.load_data import load_price_test_2026
test_prices = load_price_test_2026()

test_prices.head()

X_new = price_load.iloc[:1].copy()
X_new["residual_load"] = 9000
X_new["hour"] = 18
X_new["season"] = "winter"

print(X_new)

  Residual load actual (MW)  residual_load  price  Month  Hour  season  hour
0                   6217.86           9000  13.62      1     0  winter    18
